![trainers in a store](trainers.jpg)

Sports clothing and athleisure attire is a huge industry, worth approximately [$193 billion in 2021](https://www.statista.com/statistics/254489/total-revenue-of-the-global-sports-apparel-market/) with a strong growth forecast over the next decade! 

In this notebook, you will undertake the role of a product analyst for an online sports clothing company. The company is specifically interested in how it can improve revenue. You will dive into product data such as pricing, reviews, descriptions, and ratings, as well as revenue and website traffic, to produce recommendations for its marketing and sales teams.  

You've been provided with four datasets to investigate:

#  brands.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `brand` | Brand of the product | 

# finance.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `listing_price` | Original price of the product | 
| `sale_price` | Discounted price of the product |
| `discount` | Discount off the listing price, as a decimal | 
| `revenue` | Revenue generated by the product |

# info.csv

| Columns | Description |
|---------|-------------|
| `product_name` | Name of the product | 
| `product_id` | Unique product identifier |
| `description` | Description of the product |

# reviews.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `rating` | Average product rating | 
| `reviews` | Number of reviews for the product |

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.width", 1000)

brands = pd.read_csv("brands.csv") 
finance = pd.read_csv("finance.csv")
info = pd.read_csv("info.csv")
reviews = pd.read_csv("reviews.csv")

In [5]:
print(brands.columns)
print(finance.columns)
print(info.columns)
print(reviews.columns)

Index(['product_id', 'brand'], dtype='object')
Index(['product_id', 'listing_price', 'sale_price', 'discount', 'revenue'], dtype='object')
Index(['product_name', 'product_id', 'description'], dtype='object')
Index(['product_id', 'rating', 'reviews'], dtype='object')


To complete the project, you will need to merge the datasets, drop null values, and answer the questions below.

In [6]:
merged_df = info.merge(finance, on="product_id")
merged_df = merged_df.merge(reviews, on="product_id")
merged_df = merged_df.merge(brands, on="product_id")
merged_df.dropna(inplace=True)

print(merged_df.head())

                                        product_name product_id                                        description  listing_price  sale_price  discount  revenue  rating  reviews   brand
1               Women's adidas Originals Sleek Shoes     G27341  A modern take on adidas sport heritage, tailor...          75.99       37.99       0.5  1641.17     3.3     24.0  Adidas
2                  Women's adidas Swim Puka Slippers     CM0081  These adidas Puka slippers for women's come wi...           9.99        5.99       0.4   398.93     2.6     37.0  Adidas
3   Women's adidas Sport Inspired Questar Ride Shoes     B44832  Inspired by modern tech runners, these women's...          69.99       34.99       0.5  2204.37     4.1     35.0  Adidas
4           Women's adidas Originals Taekwondo Shoes     D98205  This design is inspired by vintage Taekwondo s...          79.99       39.99       0.5  5182.70     3.5     72.0  Adidas
5  Women's adidas Sport Inspired Duramo Lite 2.0 ...     B75586  Refin

- What is the volume of products and average revenue for Adidas and Nike products based on listing price quartiles?
    - Label products priced up to quartile one as `"Budget"`, quartile two as `"Average"`, quartile three as `"Expensive"`, and quartile four as `"Elite"`.
    - Store as a `pandas` DataFrame called `adidas_vs_nike` containing columns: `"brand"`, `"price_label"`, `"num_products"`, and `"mean_revenue"`. All numeric values should be rounded to two decimal places.

In [7]:
merged_df["price_label"] = pd.qcut(merged_df["listing_price"], q=4, labels=["Budget", "Average", "Expensive", "Elite"])

adidas_vs_nike = merged_df.groupby(["brand", "price_label"], as_index=False).agg(
    num_products=("price_label", "count"), 
    mean_revenue=("revenue", "mean")
).round(2)

print(adidas_vs_nike)

    brand price_label  num_products  mean_revenue
0  Adidas      Budget           574       2015.68
1  Adidas     Average           655       3035.30
2  Adidas   Expensive           759       4621.56
3  Adidas       Elite           587       8302.78
4    Nike      Budget           357       1596.33
5    Nike     Average             8        675.59
6    Nike   Expensive            47        500.56
7    Nike       Elite           130       1367.45


- Do any differences exist between the word count of a product's description and its mean rating?
    - Split product description length into bins of 100-character intervals and calculate the average rating and total number of reviews.
    - Store the results as a `pandas` DataFrame called `description_lengths` containing columns: `"description_length"`, `"mean_rating"`, `"total_reviews"`. Again, round numeric values to two decimal places.

In [8]:
merged_df["description_length"] = merged_df["description"].str.len()

lengthes = [0, 100, 200, 300, 400, 500, 600, 700]
labels = ["100", "200", "300", "400", "500", "600", "700"]

merged_df["description_length"] = pd.cut(merged_df["description_length"], bins=lengthes, labels=labels)

description_lengths = merged_df.groupby("description_length", as_index=False).agg(
    mean_rating=("rating", "mean"), 
    total_reviews=("reviews", "sum")
).round(2)

print(description_lengths)

  description_length  mean_rating  total_reviews
0                100         2.26           36.0
1                200         3.19        17719.0
2                300         3.28        76115.0
3                400         3.29        28994.0
4                500         3.35         4984.0
5                600         3.12          852.0
6                700         3.65          818.0
